In [1]:
import pandas as pd

In [2]:
clean_application_df = pd.read_csv("clean_application_data.csv")
clean_contracts_df = pd.read_csv("clean_contracts_data.csv")
clean_crb_df = pd.read_csv("clean_crb_data.csv")
clean_snapshot_df = pd.read_csv("clean_snapshot_summary.csv")
clean_current_savings_df = pd.read_csv("clean_current_savings.csv")

# Key Design Decisions Before Merging
## What should each row represent?
### We'll model loan default at the contract level.


### Contract_ID ties together loan performance (ContractSnapshot), borrower profile (ApplicationData), and financial behavior (Current and Savings Account Data).

### The most granular and predictive view of default is at the contract level, not just application level.



# Target variable come from?
## We don’t have an explicit Loan_Default column, but we can create it from:

## ContractSnapshot → If Contract Status is 'Default' or Number of Days Past Due_max exceeds a threshold (e.g., 90), then it's a default. We'll engineer it during the merge.

## -----------------------------------------------------------------------

# Merge Contracts + ContractSnapshot
## Merge on Contract_ID to bring in performance data.

In [3]:
contracts_full = clean_contracts_df.merge(
    clean_snapshot_df,
    on="Contract_ID",
    how="left"
)

## We use left join to retain all known contracts.

### Snapshot summary features like Outstanding Amount, Due Amount, and Days Past Due now attached.

# ---------------------------------------------------------

# Create Target Variable: Loan_Default

# Loan_Default = 1 if
## Contract_Status is "NonAccrual", "Default", etc.
## OR Days Past Due > 90

In [4]:
contracts_full['Loan_Default'] = contracts_full['Contract Status_<lambda>'].apply(
    lambda x: 1 if x in ['Default', 'NonAccrual', 'WriteOff'] else 0
)

contracts_full.loc[
    contracts_full['Number of Days Past Due_max'] > 90,
    'Loan_Default'
] = 1

# We will use it as a binary target variable.

# --------------------------------------------------------------------

# Merge contracts_full + application_df

In [5]:
merged_df = contracts_full.merge(
    clean_application_df,
    on="Application_ID",
    how="left"
)

## This brings in all borrower profile and engineered demographic features (age bin, income log, etc.)

## Now we know who took the loan.

# ------------------------------------------------------------------------------

# Merge + CRB Data

In [6]:
merged_df = merged_df.merge(
    clean_crb_df,
    on="Application_ID",
    how="left"
)

# Adds CRB Score and CRB Grade to enrich creditworthiness info

### Left join keeps all contracts even if CRB data is missing (e.g., no credit file)

# ---------------------------------------------------------------------------------------

# Aggregate Banking Behavior → Then Merge
### You should aggregate banking data by Application ID before merging.

In [7]:
# Example summary (customize as needed)
bank_agg = clean_current_savings_df.groupby("Application ID").agg({
    "Average Balance": "mean",
    "Overdraft Limit": "max",
    "Net_Cashflow": "mean",
    "Liquidity_Ratio": "mean",
    "Has_Bounced_Cheques": "max"
}).reset_index()

In [8]:
merged_df = merged_df.merge(
    bank_agg,
    left_on="Application_ID",
    right_on="Application ID",
    how="left"
).drop(columns=["Application ID"])  # avoid duplication

# Final Cleanup
## Drop unnecessary columns like Contract Status_<lambda>, Key

## Convert any date fields if needed

## Handle final missing values

## Confirm the shape and nulls

In [9]:
# Drop non-useful or leakage-prone columns
drop_cols = ['Contract Status_<lambda>', 'Key', 'Actual_Contract_End_Date']
merged_df.drop(columns=drop_cols, inplace=True, errors='ignore')

# Check shape and missing values
print("Final dataset shape:", merged_df.shape)
print(merged_df.isnull().mean().sort_values(ascending=False).head(15))

Final dataset shape: (8593, 47)
CRB Score                        0.314675
CRB Grade                        0.314675
Contract_Start_Date              0.131619
Application_Date                 0.119749
Expected_Contract_End_Date       0.058536
Overdraft Limit                  0.003957
Liquidity_Ratio                  0.000698
Number of Days Past Due_mean     0.000233
Number of Days Past Due_max      0.000233
Number of Days Past Due_count    0.000233
Due Amount_mean                  0.000233
Due Amount_max                   0.000233
Outstanding Amount_last          0.000233
Outstanding Amount_mean          0.000233
Outstanding Amount_max           0.000233
dtype: float64


In [10]:
merged_df.to_csv("loan_default_dataset.csv", index=False)

## The final dataset now has:
## 1 row = 1 loan contract

## Features from: contract, applicant, CRB, banking behavior

## Target: Loan_Default (binary)